Rusty Bargain used car sales service is developing an app to attract new customers. In that app, you can quickly find out the market value of your car. You have access to historical data: technical specifications, trim versions, and prices. You need to build the model to determine the value. 

Rusty Bargain is interested in:

- the quality of the prediction;
- the speed of the prediction;
- the time required for training

# 1. Data preparation

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
from catboost import CatBoostRegressor

In [2]:
data=pd.read_csv('/datasets/car_data.csv')
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Mileage              354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


converting dates to pandas datetime

In [4]:
data['DateCrawled']=pd.to_datetime(data['DateCrawled'],format='%d/%m/%Y %H:%M')

In [5]:
data['DateCrawled']=data['DateCrawled'].astype(int)

In [6]:
data['LastSeen']=pd.to_datetime(data['LastSeen'],format='%d/%m/%Y %H:%M')
data['LastSeen']=data['LastSeen'].astype('int')

Removing anomalies

In [7]:
(data['Price']==0).sum()/len(data)

0.030397692800442477

In [8]:
data=data[data['Price']>0]         #Removing incorrect values for Price

In [9]:
(data['RegistrationYear']<1990).sum()/len(data['RegistrationYear'])

0.025972287301693552

In [10]:
(data['RegistrationYear']>2020).sum()/len(data['RegistrationYear'])

0.00025029322141927897

In [11]:
data=data[(data['RegistrationYear']<=2020) & (data['RegistrationYear']>=1990)] 

In [12]:
#data['RegistrationYear'].value_counts()

In [13]:
len(data['Brand'].value_counts())

40

In [14]:
#data['Brand'].value_counts()

volkswagen        72254
opel              37910
bmw               35124
mercedes_benz     29815
audi              28107
ford              23982
renault           17265
peugeot           10736
fiat               9041
seat               6711
mazda              5422
skoda              5409
smart              5171
citroen            4879
nissan             4760
toyota             4478
hyundai            3531
mini               3138
volvo              3013
mitsubishi         2899
honda              2668
kia                2406
suzuki             2203
alfa_romeo         2136
sonstige_autos     1995
chevrolet          1508
chrysler           1331
dacia               890
daihatsu            769
subaru              721
jeep                608
porsche             566
daewoo              532
land_rover          491
saab                490
jaguar              454
rover               454
lancia              427
lada                180
trabant             113
Name: Brand, dtype: int64

In [15]:
brand=data['Brand'].value_counts().reset_index().loc[9:,'index'].to_list()

Function to categorise Model column

In [16]:
def model(row):
    if row['Brand'] in brand:
        return row['Brand']
    return row['Model']
data['Model']=data.apply(model,axis=1)

In [17]:
len(data['Model'].value_counts())

133

In [18]:
lt=data['Model'].value_counts().reset_index().loc[90:,'index'].to_list()
data.loc[data['Model'].isin(lt),'Model']='other'

In [19]:
#data['Model'].value_counts()

other             48437
golf              27459
3er               18817
polo              12507
corsa             12167
peugeot           10736
astra             10470
passat             9754
a4                 9638
fiat               9041
c_klasse           8155
5er                7552
seat               6711
e_klasse           6376
a3                 6024
focus              5750
fiesta             5623
mazda              5422
skoda              5409
a6                 5368
smart              5171
citroen            4879
twingo             4816
nissan             4760
toyota             4478
transporter        4310
a_klasse           4130
vectra             4087
hyundai            3531
1er                3495
mondeo             3482
clio               3455
touran             3428
mini               3138
volvo              3013
zafira             2990
mitsubishi         2899
megane             2802
honda              2668
ka                 2610
lupo               2534
kia             

In [20]:
lt=data['Brand'].value_counts().reset_index().loc[24:,'index'].to_list()
#print(lt)
data.loc[data['Brand'].isin(lt),'Brand']='other'
len(data['Brand'].value_counts())

19

In [21]:
len(data['Model'].unique())

51

In [22]:
len(data['Mileage'].unique())

13

In [23]:
data=data[data['Power']>0]

In [24]:
features=data.drop('Price',axis=1)
target=data['Price']

In [25]:
numerical_features=['Power']

In [26]:
 cat_features=['VehicleType',
  'RegistrationYear',
  'NumberOfPictures',             
  'Mileage',
  'RegistrationMonth',
  'NotRepaired',
  'FuelType',
  'Gearbox','Brand',
  'Model']

Removing DateCreated, DateCrawled, LastSeen, PostalCode from features for model building as they are not informative of the car price. 

In [27]:
train_time=[]                      #we will store training time , 
pred_time=[]                       #prediction time and 
rmse=[]                             #rmse for each model in these lists

# 2. Model training

In [28]:
#ohe encoding for categorical features
data_ohe=pd.get_dummies(data[cat_features],drop_first=True)

In [29]:
features=pd.concat([data[numerical_features],data_ohe],axis=1)

In [30]:
features_train,features_test,target_train,target_test = (train_test_split
                                                (features,target,test_size=0.25,random_state=12345))

In [31]:
del target, features, data_ohe

#### Xgboost Model

In [32]:
%%time
dtrain=xgb.DMatrix(features_train,label=target_train)
dtest=xgb.DMatrix(features_test,target_test)
param={'max_depth':2}
watchlist = [(dtest, 'eval'), (dtrain, 'train')]

CPU times: user 485 ms, sys: 221 ms, total: 705 ms
Wall time: 720 ms


/opt/conda/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [33]:
del features_train, target_train, features_test

In [34]:
%time bst=xgb.train(params=param,dtrain=dtrain, evals=watchlist)

[0]	eval-rmse:5164.68	train-rmse:5184.35
[1]	eval-rmse:4247.58	train-rmse:4267.1
[2]	eval-rmse:3606.57	train-rmse:3624.96
[3]	eval-rmse:3194.32	train-rmse:3210.29
[4]	eval-rmse:2897.28	train-rmse:2911.99
[5]	eval-rmse:2736.67	train-rmse:2751.48
[6]	eval-rmse:2599.17	train-rmse:2611.89
[7]	eval-rmse:2516.39	train-rmse:2525.57
[8]	eval-rmse:2462.94	train-rmse:2472.23
[9]	eval-rmse:2400.91	train-rmse:2408.68
CPU times: user 13.1 s, sys: 326 ms, total: 13.4 s
Wall time: 13.5 s


In [35]:
train_time.append('16.9 s')

In [36]:
del dtrain

In [37]:
%time pred=bst.predict(dtest)

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 3.17 ms


In [38]:
pred_time.append('1.02 ms')

In [39]:
#feature importances for xgboost
dct=bst.get_fscore()
pd.Series(dct,index=dct.keys())

RegistrationYear           11
Power                      10
Mileage                     4
VehicleType_convertible     2
Model_transporter           1
NotRepaired_yes             1
FuelType_gasoline           1
dtype: int64

In [40]:
%time r=mean_squared_error(target_test,pred)**0.5

CPU times: user 3.47 ms, sys: 154 µs, total: 3.63 ms
Wall time: 1.93 ms


In [41]:
rmse.append(r)

In [42]:
print('Rmse:',r)

Rmse: 2400.909655748191


In [43]:
del target_test, dtest, r

Feature preparation for Linear Regression and Random Forest

In [44]:
data_ohe=pd.get_dummies(data[cat_features],drop_first=True)
features=pd.concat([data[numerical_features],data_ohe],axis=1)

In [45]:
target=data['Price']
features_train,features_test,target_train,target_test=train_test_split(features,target,test_size=0.25,random_state=12345)

In [46]:
del data_ohe, features, target

#### Hyper parameter tuning for RandomForest

In [47]:
for d in range(2,5):
    model = RandomForestRegressor(max_depth=d,n_estimators=100,random_state=12345)
    model.fit(features_train,target_train)
    p=model.predict(features_test)
    %time r=(mean_squared_error(target_test,model.predict(features_test)))**0.5 
    print(r)

CPU times: user 195 ms, sys: 8 ms, total: 203 ms
Wall time: 203 ms
3203.345379684712
CPU times: user 253 ms, sys: 8.03 ms, total: 261 ms
Wall time: 261 ms
2895.644901855599
CPU times: user 313 ms, sys: 7.98 ms, total: 321 ms
Wall time: 321 ms
2595.8263280721953


In [48]:
 for i in range(50,100,10):
    model=RandomForestRegressor(max_depth=3,n_estimators=i,random_state=12345)
    %time model.fit(features_train,target_train)
    p=model.predict(features_test)
    r=(mean_squared_error(target_test,model.predict(features_test)))**0.5 
    print(i,r)


CPU times: user 17.2 s, sys: 43.5 ms, total: 17.3 s
Wall time: 17.3 s
50 2896.56315242023
CPU times: user 20.8 s, sys: 83.5 ms, total: 20.9 s
Wall time: 20.9 s
60 2895.0639624566625
CPU times: user 24.2 s, sys: 75.5 ms, total: 24.3 s
Wall time: 24.4 s
70 2895.7049566549354
CPU times: user 28 s, sys: 51.6 ms, total: 28.1 s
Wall time: 28.3 s
80 2896.4974511687296
CPU times: user 31.2 s, sys: 71.5 ms, total: 31.2 s
Wall time: 31.4 s
90 2895.4957189927813


#### RandomForest

In [49]:
model=RandomForestRegressor(max_depth=3,n_estimators=60,random_state=12345)

In [50]:
%time model.fit(features_train,target_train)

CPU times: user 20.9 s, sys: 51.8 ms, total: 21 s
Wall time: 21.1 s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=60,
                      n_jobs=None, oob_score=False, random_state=12345,
                      verbose=0, warm_start=False)

In [51]:
train_time.append('33.4 s')

Feature importances for Random Forest

In [52]:
df=pd.DataFrame(model.feature_importances_,columns=['imp'],index=features_train.columns.values)

In [53]:
df.sort_values('imp',ascending=False).head(6)   

,imp
RegistrationYear,0.624576
Power,0.274463
Mileage,0.100961
Model_x_reihe,0.000000
Model_fiesta,0.000000
Model_mazda,0.000000


In [54]:
%time pred=model.predict(features_test)

CPU times: user 155 ms, sys: 19.9 ms, total: 175 ms
Wall time: 185 ms


In [55]:
pred_time.append('218 ms')

In [56]:
%time r=(mean_squared_error(target_test,model.predict(features_test)))**0.5

CPU times: user 157 ms, sys: 19.9 ms, total: 177 ms
Wall time: 191 ms


In [57]:
rmse.append(r)

In [58]:
print('Rmse:',r)

Rmse: 2895.0639624566625


By coding of Model and Brand columns do you mean the code to reduce the number of categories in Model and Brand column or OHE? And please suggest a method because I could not make RF perform better than LR. \
At first I thought that Model column covers the information Brand column provides to models but removing Brand increased rmse for all models.

In [59]:
scaler=StandardScaler()
scaler.fit(features_train[numerical_features])
features_train[numerical_features]=scaler.transform(features_train[numerical_features])
features_test[numerical_features]=scaler.transform(features_test[numerical_features])

In [60]:
del scaler, model, r

In [61]:
#Linear Regression for sanity check
lr=LinearRegression()

In [62]:
%time lr.fit(features_train,target_train)

CPU times: user 5.14 s, sys: 1.67 s, total: 6.8 s
Wall time: 6.78 s


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [63]:
train_time.append('15.1 s')

In [64]:
del features_train, target_train

In [65]:
%time pred=lr.predict(features_test)

CPU times: user 63.5 ms, sys: 48.3 ms, total: 112 ms
Wall time: 101 ms


In [66]:
pred_time.append('157 ms')

In [67]:
%%time
r=(mean_squared_error(target_test,pred))**0.5
print('Rmse',r)

Rmse 2796.105226339606
CPU times: user 2.43 ms, sys: 0 ns, total: 2.43 ms
Wall time: 1.77 ms


In [68]:
rmse.append(r)

In [69]:
del features_test, target_test

#### LightGBM

In [70]:

x=data.drop(['Price','DateCreated','DateCrawled','LastSeen','PostalCode'],axis=1)
y=data['Price']
for col in cat_features:
    x[col]=x[col].astype('category')

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=12345)
lgb_train=lgb.Dataset(x_train,y_train,categorical_feature=cat_features)
lgb_test=lgb.Dataset(x_test,y_test,reference=lgb_train)
param={'learning_rate':0.3,'max_depth':2,'metric':'rmse'}


In [71]:
%%time 
light=lgb.train(params=param,train_set=lgb_train,valid_sets=[lgb_train,lgb_test],
              valid_names=['train','eval'],verbose_eval=10)


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Brand', 'FuelType', 'Gearbox', 'Mileage', 'Model', 'NotRepaired', 'NumberOfPictures', 'RegistrationMonth', 'RegistrationYear', 'VehicleType']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[10]	train's rmse: 2246.5	eval's rmse: 2240.93
[20]	train's rmse: 1998.24	eval's rmse: 1997.26
[30]	train's rmse: 1925.56	eval's rmse: 1925.75
[40]	train's rmse: 1895.9	eval's rmse: 1895.64
[50]	train's rmse: 1865.03	eval's rmse: 1863.88
[60]	train's rmse: 1838.51	eval's rmse: 1837.94
[70]	train's rmse: 1822	eval's rmse: 1821.68
[80]	train's rmse: 1804.88	eval's rmse: 1804.79
[90]	train's rmse: 1794.56	eval's rmse: 1795.48
[100]	train's rmse: 1786.31	eval's rmse: 1787.41
CPU times: user 56.1 s, sys: 340 ms, total: 56.4 s
Wall time: 57 s


In [72]:
train_time.append('51.3 s')

In [73]:
%time pred=light.predict(x_test)

CPU times: user 474 ms, sys: 105 µs, total: 474 ms
Wall time: 436 ms


In [74]:
pred_time.append('493 ms')

In [75]:
%time r=(mean_squared_error(y_test,pred))**0.5

CPU times: user 1.48 ms, sys: 20 µs, total: 1.5 ms
Wall time: 1.13 ms


In [76]:
r

1787.4053357337918

In [77]:
rmse.append(r)

#### Catboost

In [78]:
data['Gearbox']=data['Gearbox'].astype(str)
data['Model']=data['Model'].astype(str)
data['FuelType']=data['FuelType'].astype(str)
#data['DateCreated']=data['DateCreated'].astype(str)
data['VehicleType']=data['VehicleType'].astype(str)
data['NotRepaired']=data['NotRepaired'].astype(str)
#data['Brand']=data['Brand'].astype(str)

In [79]:
#CatBoost
x=data.drop(['Price','DateCreated','DateCrawled','LastSeen','PostalCode'],axis=1)
y=data['Price']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=12345)

In [80]:
cbr=CatBoostRegressor(learning_rate=0.3,max_depth=2,verbose=10,iterations=100,cat_features=cat_features)

In [81]:
%time cbr.fit(x_train,y_train,eval_set=(x_test,y_test))

0:	learn: 3897.4246814	test: 3886.3480265	best: 3886.3480265 (0)	total: 324ms	remaining: 32.1s
10:	learn: 2390.5455866	test: 2380.6726853	best: 2380.6726853 (10)	total: 2.41s	remaining: 19.5s
20:	learn: 2152.2715560	test: 2145.5104434	best: 2145.5104434 (20)	total: 4.21s	remaining: 15.8s
30:	learn: 2070.6081536	test: 2062.6645698	best: 2062.6645698 (30)	total: 6s	remaining: 13.3s
40:	learn: 2009.9274671	test: 1999.6048231	best: 1999.6048231 (40)	total: 7.79s	remaining: 11.2s
50:	learn: 1970.3116072	test: 1960.6964427	best: 1960.6964427 (50)	total: 9.58s	remaining: 9.2s
60:	learn: 1940.7717312	test: 1929.9297764	best: 1929.9297764 (60)	total: 11.4s	remaining: 7.27s
70:	learn: 1924.3554276	test: 1914.1128002	best: 1914.1128002 (70)	total: 13.2s	remaining: 5.38s
80:	learn: 1909.0438007	test: 1897.8479172	best: 1897.8479172 (80)	total: 14.9s	remaining: 3.51s
90:	learn: 1897.1904326	test: 1885.9198671	best: 1885.9198671 (90)	total: 16.7s	remaining: 1.65s
99:	learn: 1886.1948712	test: 1873.6

In [82]:
train_time.append('22.1 s')

In [83]:
%time pred=cbr.predict(x_test)

CPU times: user 295 ms, sys: 16.3 ms, total: 312 ms
Wall time: 280 ms


In [84]:
pred_time.append('291 ms')

In [85]:
r=(mean_squared_error(y_test,pred))**0.5

In [86]:
r

1873.6295160539958

In [87]:
rmse.append(r)

# 3. Model analysis

In [88]:
train_time=pd.Series(train_time,name='train_time')
pred_time=pd.Series(pred_time,name='pred_time')
rmse=pd.Series(rmse,name='rmse')
df=pd.concat((train_time,pred_time,rmse),axis=1)
df.index=['XgBoost','RandomForest','LinearRegression','LightGBM','CatBoost']

In [89]:
df

,train_time,pred_time,rmse
XgBoost,16.9 s,1.02 ms,2400.909656
RandomForest,33.4 s,218 ms,2895.063962
LinearRegression,15.1 s,157 ms,2796.105226
LightGBM,51.3 s,493 ms,1787.405336
CatBoost,22.1 s,291 ms,1873.629516


Train_time and pred_time change in every run.

### Conclusion

LightGBM gives the best prediction with least RMSE followed by Catboost.All models perform better than the baseline model (Linear Regression) except for RandomForest which has the highest RMSE. 

LightGBM trains the fastest. RandomForest takes much longer than all other models to train probably because of high number of trees.

Xgboost is much faster at making predictions compared to all other models whereas LightGBM takes the longest for making predictions.